In [1]:
from download_delgado.delgado_datasets import DownloadAndConvertDelgadoDatasets
from mleap.data import Data
from mleap.estimators.estimators import instantiate_default_estimators
from mleap.experiments import Orchestrator
from mleap.analyze_results import AnalyseResults
from download_delgado.delgado_datasets import DownloadAndConvertDelgadoDatasets


/media/viktor/Data/PhD/mleap/py36/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


### Step 1: Download the datasets

In [ ]:
delgado = DownloadAndConvertDelgadoDatasets()
datasets, metadata = delgado.download_and_extract_datasets(verbose = False)

### Step 2: Define Input and Output HDF5 objects

In [2]:
data = Data()
input_io = data.open_hdf5('data/delgado.hdf5', mode='a')
out_io = data.open_hdf5('data/classification.hdf5', mode='a')

### Step 3: Save the datasets in HDF5 database

In [ ]:
# files io object needs to be used in test orch
# returned object needs to be integrated in code

data.pandas_to_db(save_loc_hdf5='delgado_datasets/', datasets=datasets, 
                  dts_metadata=metadata, input_io=input_io)

### Step 4: Split datasets

In [3]:
dts_names_list, dts_names_list_full_path = data.list_datasets(hdf5_io=input_io, hdf5_group='delgado_datasets/')
split_dts_list = data.split_datasets(hdf5_in=input_io, hdf5_out=out_io, dataset_paths=dts_names_list_full_path)

2018-03-07 22:51:22,551 [MainThread  ] [WARNI]  Skipping abalone as test/train split already exists in output h5 file.
2018-03-07 22:51:22,561 [MainThread  ] [WARNI]  Skipping acute_inflammation as test/train split already exists in output h5 file.
2018-03-07 22:51:22,585 [MainThread  ] [WARNI]  Skipping acute_nephritis as test/train split already exists in output h5 file.
2018-03-07 22:51:22,612 [MainThread  ] [WARNI]  Skipping adult as test/train split already exists in output h5 file.
2018-03-07 22:51:22,639 [MainThread  ] [WARNI]  Skipping annealing as test/train split already exists in output h5 file.
2018-03-07 22:51:22,658 [MainThread  ] [WARNI]  Skipping arrhythmia as test/train split already exists in output h5 file.
2018-03-07 22:51:22,684 [MainThread  ] [WARNI]  Skipping audiology_std as test/train split already exists in output h5 file.
2018-03-07 22:51:22,710 [MainThread  ] [WARNI]  Skipping balance_scale as test/train split already exists in output h5 file.
2018-03-07 22:

2018-03-07 22:51:23,929 [MainThread  ] [WARNI]  Skipping nursery as test/train split already exists in output h5 file.
2018-03-07 22:51:23,958 [MainThread  ] [WARNI]  Skipping oocytes_merluccius_nucleus_4d as test/train split already exists in output h5 file.
2018-03-07 22:51:23,976 [MainThread  ] [WARNI]  Skipping oocytes_merluccius_states_2f as test/train split already exists in output h5 file.
2018-03-07 22:51:23,996 [MainThread  ] [WARNI]  Skipping oocytes_trisopterus_nucleus_2f as test/train split already exists in output h5 file.
2018-03-07 22:51:24,010 [MainThread  ] [WARNI]  Skipping oocytes_trisopterus_states_5b as test/train split already exists in output h5 file.
2018-03-07 22:51:24,025 [MainThread  ] [WARNI]  Skipping optical as test/train split already exists in output h5 file.
2018-03-07 22:51:24,045 [MainThread  ] [WARNI]  Skipping ozone as test/train split already exists in output h5 file.
2018-03-07 22:51:24,057 [MainThread  ] [WARNI]  Skipping page_blocks as test/trai

### Step 5: Instantiate estimator objects and the experiments orchestrator class.

In [4]:
instantiated_models = instantiate_default_estimators(estimators=['SVC'])

test_o = Orchestrator(hdf5_input_io=input_io, hdf5_output_io=out_io, dts_names=dts_names_list,
                 original_datasets_group_h5_path='delgado_datasets/')

### Step 6: Run the experiments

In [6]:
test_o.run(modelling_strategies=instantiated_models)

*** Training models on dataset: abalone. Total datasets processed: 1/121 ***
*** Training models on dataset: acute_inflammation. Total datasets processed: 2/121 ***
*** Training models on dataset: acute_nephritis. Total datasets processed: 3/121 ***
*** Training models on dataset: adult. Total datasets processed: 4/121 ***
***************************
********  EXITING  ********
***************************


SystemExit: 

/media/viktor/Data/PhD/mleap/py36/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2918: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


### Step 6: Analyze the results

In [ ]:
analyze = AnalyseResults(hdf5_output_io=out_io, hdf5_input_io=input_io)
observations = analyze.calculate_loss_all_datasets(input_h5_original_datasets_group='delgado_datasets/', 
                                    output_h5_predictions_group='experiments/predictions/', 
                                    metric='mean_squared_error')

#### t-test

In [ ]:
t_test, t_test_df = analyze.t_test(observations)
t_test_df

#### sign test

In [ ]:
sign_test, sign_test_df = analyze.sign_test(observations)
sign_test_df

#### t-test with bonferroni correction

In [ ]:
t_test_bonferroni, t_test_bonferroni_df = analyze.t_test_with_bonferroni_correction(observations)
t_test_bonferroni_df

#### Wilcoxon test

In [ ]:
wilcoxon_test, wilcoxon_test_df = analyze.wilcoxon_test(observations)
wilcoxon_test_df

#### Friedman test

In [ ]:
friedman_test, friedman_test_df = analyze.friedman_test(observations)
friedman_test_df

#### Nemenyi test

In [ ]:
nemeniy_test = analyze.nemenyi(observations)
nemeniy_test